In [12]:
from langchain_openai import ChatOpenAI
from langchain.callbacks.tracers import ConsoleCallbackHandler
from os import environ

In [29]:
model = ChatOpenAI(model="gpt-4o", api_key=environ['OPENAI_API_KEY'])
#model = ChatOpenAI(model="gpt-4o", api_key=environ['OPENAI_API_KEY']).with_config({'callbacks': [ConsoleCallbackHandler()]})

In [30]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None}, id='run-c15ad46e-5896-48f4-adf0-f78c6f1e6f25-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21})

In [31]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I don't have access to personal information about users, so I don't know your name. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 11, 'total_tokens': 35}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None}, id='run-af881f38-1bf5-465d-97e7-cd629f9746ab-0', usage_metadata={'input_tokens': 11, 'output_tokens': 24, 'total_tokens': 35})

In [32]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='You mentioned that your name is Bob. How can I help you today, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 33, 'total_tokens': 50}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None}, id='run-ee918e88-ccd2-4922-9719-876171145bf2-0', usage_metadata={'input_tokens': 33, 'output_tokens': 17, 'total_tokens': 50})

In [33]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [34]:
config = {"configurable": {"session_id": "abc2"}}

In [35]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

'Hello, Bob! How can I assist you today?'

In [36]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'You mentioned that your name is Bob. How can I help you today, Bob?'

In [37]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, but I don't have access to personal information about you. How can I assist you today?"

In [38]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"You told me earlier that your name is Bob. Is there something specific you'd like to discuss or ask about?"

In [39]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [40]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

'Hi Bob! How can I assist you today?'

In [41]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [42]:
config = {"configurable": {"session_id": "abc5"}}

In [43]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

'Hi Jim! How can I assist you today?'

In [44]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jim! How can I help you today?'

In [45]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [46]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "pirate"}
)

response.content

"Ahoy, Bob! Welcome aboard, matey! How can this ol' sea dog be of service to ye today?"

In [47]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [48]:
config = {"configurable": {"session_id": "abc11"}}

In [49]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'¡Hola, Todd! ¿Cómo estás? ¿En qué puedo ayudarte hoy?'

In [50]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=config,
)

response.content

'Tu nombre es Todd. ¿Hay algo más en lo que pueda ayudarte?'

In [51]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [52]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, but I don't know your name. You haven't shared it with me yet."

In [53]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked, "What\'s 2 + 2?"'

In [54]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [55]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"You haven't provided your name yet. What is your name?"

In [56]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You haven't asked a math problem yet. If you have one, feel free to ask, and I'll do my best to help you with it!"

### Conversation Summary

In [57]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import OpenAI

In [60]:
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
llm = OpenAI(temperature=0)
conversation_with_summary = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=OpenAI()),
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")

/tmp/ipykernel_4087/1994856562.py:4: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  conversation_with_summary = ConversationChain(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


' Hello! I am currently running on a server in a data center in California. The temperature in the room is 72 degrees Fahrenheit and the humidity is at 45%. My processors are running at 80% capacity and I am processing data from various sources such as social media, news articles, and weather reports. Is there something specific you would like to know?'

In [61]:
conversation_with_summary.predict(input="Tell me more about it!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human greets the AI and asks what it is currently doing. The AI responds, stating it is running on a server in California and providing specific details about its environment and tasks. The AI also asks if there is anything the human would like to know.
Human: Tell me more about it!
AI:

> Finished chain.


' Of course! Currently, I am running on a server located in California. The server is equipped with high-speed processors and a large amount of memory, allowing me to process and analyze vast amounts of data in a short amount of time. I am constantly receiving and processing information from various sources, such as sensors, databases, and user interactions. My main task is to provide accurate and helpful responses to user inquiries, like the one you just asked. Is there anything else you would like to know?'

In [62]:
conversation_with_summary.predict(input="Very cool -- what is the scope of the project?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human greets the AI and asks what it is currently doing. The AI responds by providing specific details about its environment and tasks, mentioning that it is running on a server in California equipped with high-speed processors and a large amount of memory. It also states that its main task is to provide accurate and helpful responses to user inquiries. The AI then asks if there is anything else the human would like to know.
Human: Very cool -- what is the scope of the project?
AI:

> Finished chain.


' Thank you! The scope of the project is quite broad, as I am designed to handle a wide range of inquiries and tasks. Some of my main functions include providing information, assisting with tasks, and learning from user interactions to improve my responses. Is there anything specific you would like to know about the project?'